<a href="https://colab.research.google.com/github/pearlcrum/Test/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_week03_assignment_(%EA%B9%80%EC%A7%80%EB%AF%BC)%EC%9D%98_%EC%82%AC%EB%B3%B8(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2022.06.12

---

# 머신러닝(GSI7645.01-00)
## [week 03] 지도학습1
---
### Spaceship titanic
* kaggle 결과 제출
---

<h1> Spaceship titanic </h1>

<img src="https://velog.velcdn.com/images/lifeisbeautiful/post/386322a9-5edf-437f-a3aa-39ba929b3414/image.jpg" alt="post-thumbnail" class="sc-eGRUor hxxOgP">

<p><a href="https://www.kaggle.com/competitions/spaceship-titanic">https://www.kaggle.com/competitions/spaceship-titanic</a></p>

<h2 id="span-stylecolorrgb1-124-200-데이터-구조-span"><span style="color:rgb(1, 124, 200)"> <strong>데이터 구조</strong> </span></h2>

<ul>
<li>
<p><strong>train.csv</strong> - 승객의 약 3분의 2(~8700명)에 대한 개인 기록으로, 훈련 데이터로 사용됩니다.</p>
<ul>
<li><code>PassengerId</code> : 각 승객의 고유 ID입니다. 각 Id는 <code>ggg_pp</code> 형식을 취합니다. 여기서 <code>gggg</code>는 승객과 함께 여행하는 그룹을 나타내고 pp는 그룹 내에서 승객의 번호입니다. 그룹의 사람들은 종종 가족 구성원이지만 항상 그렇지는 않습니다.</li>
<li><code>HomePlanet</code> : 승객이 출발한 행성, 일반적으로 영구 거주지의 행성입니다.</li>
<li><code>CryoSleep</code> : 승객이 항해 중 수면상태로 전환되도록 선택했는지 여부를 나타냅니다. 극저온 수면 중인 승객들은 객실에 갇혀 있습니다.<br>
- <code>Cabin</code> : 승객이 묵고 있는 객실 번호. <code>deck/num/side</code> 형태를 취합니다. 여기서 <code>side</code>는 좌현의 경우 <code>P</code>, 우현의 경우 <code>S</code>일 수 있습니다.<br>
- <code>Destination</code> : 승객이 내리는 도착지.</li>
<li><code>Age</code> : 승객의 나이</li>
<li><code>VIP</code> : 승객이 항해 중 특별 VIP 서비스에 대한 비용을 지불했는지 여부</li>
<li><code>RoomService</code>, <code>FoodCourt</code>, <code>ShoppingMall</code>, <code>Spa</code>, <code>VRDeck</code> : 승객들이 우주선 타이타닉의 많은 고급 편의시설 각각에 청구한 금액입니다.</li>
<li><code>Name</code> : 승객의 성 및 이름</li>
<li><code>Transported</code> : 승객이 다른 차원으로 운송되었는지 여부. 예측하려는 열인 목표값입니다.</li>
</ul>
</li>
<li>
<p><strong>test.csv</strong> : 남은 승객의 1/3(약 4300명)에 대한 개인 기록으로, 테스트 데이터로 사용됩니다. 이 세트의 승객에 대한 Transported 값을 예측하는 것이 작업입니다.</p>
</li>
<li>
<p><strong>sample_submission.csv</strong> - 제출 파일 형식의 예시입니다.</p>
<ul>
<li><code>PassengerId</code> : 각 승객에 대한 ID입니다.</li>
<li><code>Transported</code> : 목표값 입니다. 승객에 따라<code>True</code> 또는 <code>False</code> 를 예측합니다.</li>
</ul>
</li>
</ul>

In [107]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
%config IPCompleter.use_jedi = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.
  """


## 0. kaggle

In [2]:
## B. 환경변수에 등록

import os

os.environ['KAGGLE_USERNAME'] = 'jamiekeem'
os.environ['KAGGLE_KEY'] = '3fe7e3478d4f09a5ecb64d52cab37972'

# 1. 문제 정의
## 1.1 download dataset

In [3]:
! kaggle competitions download -c spaceship-titanic -w
! unzip ./spaceship-titanic.zip

  0% 0.00/299k [00:00<?, ?B/s]
100% 299k/299k [00:00<00:00, 87.1MB/s]
Archive:  ./spaceship-titanic.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [150]:
! ls ./*.csv

./sample_submission.csv  ./test.csv  ./train.csv


## 1.2 inspect dataset

In [151]:
import pandas as pd

df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')


## On Your Own
# count records (shape)
print("(train): {}".format(df_train.shape))  # print shape
print("(test): {}".format(df_test.shape))    # print shape

# count features & labels
id_columns = ['PassengerId']
label_columns = ['Transported']
feature_columns = [c for c in df_train.columns if c not in id_columns + label_columns]
# list comph: df_train의 columns 중에서 id_columns와 label_columns가 아닌 컬럼들을 전부 불러와라
print("# of id_columns: {} ({})".format(len(id_columns), id_columns))
print("# of label_columns: {} ({})".format(len(label_columns), label_columns))
print("# of feature_columns: {} ({})".format(len(feature_columns), feature_columns))

(train): (8693, 14)
(test): (4277, 13)
# of id_columns: 1 (['PassengerId'])
# of label_columns: 1 (['Transported'])
# of feature_columns: 12 (['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name'])


# 2. EDA

In [152]:
# df_train 의 상위 5개 행을 확인해주세요

df_train.head(5)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [153]:
# df_train 데이터의 행과 열의 개수를 확인해주세요
df_train.shape

(8693, 14)

In [154]:
# df_train 데이터의 정보와 타입을 확인해주세요
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


[평가 종합]

* Tranported 변수는 bool 변수라서, 수치형으로 변경하지 않아도 된다.
* object 변수가 많다 -> label, one-hot encoder로 변환 필요함!

Name은 필요없는 변수라 생각 DROP

In [155]:
df_train=df_train.drop(columns='Name')

In [156]:
df_test=df_test.drop(columns='Name')

In [157]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(6)
memory usage: 823.6+ KB


In [158]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
dtypes: float64(6), object(6)
memory usage: 401.1+ KB


In [159]:
object_type = ['PassengerId','HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']

for column in object_type:
    a = df_train[column].unique()
    print(column, ':\t', a)

# "컬럼 이름: [unique로 확인한 변수 구성]"

PassengerId :	 ['0001_01' '0002_01' '0003_01' ... '9279_01' '9280_01' '9280_02']
HomePlanet :	 ['Europa' 'Earth' 'Mars' nan]
CryoSleep :	 [False True nan]
Cabin :	 ['B/0/P' 'F/0/S' 'A/0/S' ... 'G/1499/S' 'G/1500/S' 'E/608/S']
Destination :	 ['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan]
VIP :	 [False True nan]


[평가]
* 'CryoSleep', 'VIP'는 bool 형태 또는 수치형으로 바꿀 필요가 있다.
* HomePlanet, Destination은 label -> one-hot 인코딩을 해야할 것 같다
* Cabin은 ... 뭐지?

In [160]:
# df_train 에서 object type 변수의 구성을 확인해봅시다

a= df_train['Cabin'].unique()
b= df_train['Destination'].unique()
print(a)
print(b)

['B/0/P' 'F/0/S' 'A/0/S' ... 'G/1499/S' 'G/1500/S' 'E/608/S']
['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan]


In [161]:
# Cabin 변수의 유일값들은 모두 몇개일까요?
# Cabin : 승객이 묵고 있는 객실 번호. deck/num/side 형태를 취합니다.
# 여기서 side는 좌현의 경우 P, 우현의 경우 S일 수 있습니다.

len(df_train['Cabin'].unique())

6561

In [162]:
# Deck 변수를 만들어주세요

df_train['Deck'] = df_train['Cabin'].str[0]
df_train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F


In [163]:
df_test['Deck'] = df_test['Cabin'].str[0]
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,G
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,F
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,C
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,C
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,F


In [164]:
# Side 변수를 만들어주세요
df_train['Side'] = df_train['Cabin'].str[-1]
df_train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Side
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,P
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,S
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,S
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,S
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,S


In [165]:
df_test['Side'] = df_test['Cabin'].str[-1]
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Side
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,G,S
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,F,S
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,C,S
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,C,S
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,F,S


In [166]:
# Cabin 변수를 삭제해주세요
df_train.drop(['Cabin'], inplace=True, axis=1)
df_train.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Side
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,P
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,S
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,S
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,S
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,S


In [167]:
df_test.drop(['Cabin'], inplace=True, axis=1)
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Side
0,0013_01,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,G,S
1,0018_01,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,F,S
2,0019_01,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,C,S
3,0021_01,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,C,S
4,0023_01,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,F,S


In [168]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Destination   8511 non-null   object 
 4   Age           8514 non-null   float64
 5   VIP           8490 non-null   object 
 6   RoomService   8512 non-null   float64
 7   FoodCourt     8510 non-null   float64
 8   ShoppingMall  8485 non-null   float64
 9   Spa           8510 non-null   float64
 10  VRDeck        8505 non-null   float64
 11  Transported   8693 non-null   bool   
 12  Deck          8494 non-null   object 
 13  Side          8494 non-null   object 
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [169]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Destination   4185 non-null   object 
 4   Age           4186 non-null   float64
 5   VIP           4184 non-null   object 
 6   RoomService   4195 non-null   float64
 7   FoodCourt     4171 non-null   float64
 8   ShoppingMall  4179 non-null   float64
 9   Spa           4176 non-null   float64
 10  VRDeck        4197 non-null   float64
 11  Deck          4177 non-null   object 
 12  Side          4177 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [170]:
# object type 변수의 구성을 다시 확인해봅시다

object_type = ['PassengerId','HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']

for column in object_type:
    a = df_train[column].unique()
    print(column, ':\t', a)

# "컬럼 이름: [unique로 확인한 변수 구성]"

PassengerId :	 ['0001_01' '0002_01' '0003_01' ... '9279_01' '9280_01' '9280_02']
HomePlanet :	 ['Europa' 'Earth' 'Mars' nan]
CryoSleep :	 [False True nan]
Destination :	 ['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan]
VIP :	 [False True nan]
Deck :	 ['B' 'F' 'A' 'G' nan 'E' 'D' 'C' 'T']
Side :	 ['P' 'S' nan]


[평가]
* 'CryoSleep', 'VIP'는 bool 형태 또는 수치형으로 바꿀 필요가 있다.
* HomePlanet, Destination은 label -> one-hot 인코딩을 해야할 것 같다
* Cabin은 해결
    * Deck 생성 -> 인코딩 필요
    * Side 생성 -> 인코딩 필요

In [171]:
## 결측치가 있는 컬럼은 무엇인가요??

## 결측치가 얼만큼 존재하나요??

# df_train.isna().sum()

## 결측치가 있는 변수를 N 리스트로 생성해주세요

N=[]
for col in df_train.columns:
    a = df_train[col].isna().sum()
    if a != 0:
        N.append(col)

print(N)

['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Deck', 'Side']


In [172]:
M=[]
for col in df_test.columns:
    a = df_test[col].isna().sum()
    if a != 0:
        M.append(col)

print(M)

['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Deck', 'Side']


결측치를 어떻게 처리하는 것이 좋을까요?
* 빈도수가 가장 많은 변수 또는 평균값으로 대체한다 => df_train_1
* 다 없앤다 -> 단점: 데이터가 손실될 위험이 있음 => df_train_2

(1) 빈도수가 가장 많은 변수 또는 평균값으로 대체한 것을 df_train_1 로 생성한다

In [173]:
# df_train 에서 결측치가 있는 변수 리스트 N 을 활용해
# 결측치가 있는 변수들의 평균을 알아봅시다
# (힌트: describe 사용)

df_train[N].describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [174]:
df_train[N].describe()['Age']['mean']

28.82793046746535

In [175]:
# 위에서 확인한 변수를 key, 평균값을 value로 가지는 딕셔너리 re 를 생성해봅시다
# 단, 평균값은 소수점 2째 자리까지 생성합시다

re = {}
for col in df_train[N].describe().columns:
    a = df_train[N].describe()[col]['mean']
    a = round(a, 2)
    re[col] = a
print(re)

{'Age': 28.83, 'RoomService': 224.69, 'FoodCourt': 458.08, 'ShoppingMall': 173.73, 'Spa': 311.14, 'VRDeck': 304.85}


In [176]:
fe = {}
for col in df_test[N].describe().columns:
    a = df_test[N].describe()[col]['mean']
    a = round(a, 2)
    fe[col] = a
print(fe)

{'Age': 28.66, 'RoomService': 219.27, 'FoodCourt': 439.48, 'ShoppingMall': 177.3, 'Spa': 303.05, 'VRDeck': 310.71}


In [177]:
# df_train에서 결측치가 있는 변수 리스트 N 을 활용해
# 결측치가 있는 변수들의 최빈값을 알아봅시다
# (힌트: describe exclude=np.number 사용)

import numpy as np
df_train.describe(exclude=np.number)

,PassengerId,HomePlanet,CryoSleep,Destination,VIP,Transported,Deck,Side
count,8693,8492,8476,8511,8490,8693,8494,8494
unique,8693,3,2,3,2,2,8,2
top,0001_01,Earth,False,TRAPPIST-1e,False,True,F,S
freq,1,4602,5439,5915,8291,4378,2794,4288


In [178]:
# 위에서 확인한 변수를 key, 최빈값을 value로 설정해
# re 딕셔너리에 추가해봅시다

for col in df_train.describe(exclude=np.number).columns:
    a = df_train.describe(exclude=np.number)[col]['top']
    re[col] = a
print(re)

{'Age': 28.83, 'RoomService': 224.69, 'FoodCourt': 458.08, 'ShoppingMall': 173.73, 'Spa': 311.14, 'VRDeck': 304.85, 'PassengerId': '0001_01', 'HomePlanet': 'Earth', 'CryoSleep': False, 'Destination': 'TRAPPIST-1e', 'VIP': False, 'Transported': True, 'Deck': 'F', 'Side': 'S'}


In [179]:
for col in df_test.describe(exclude=np.number).columns:
    a = df_train.describe(exclude=np.number)[col]['top']
    fe[col] = a
print(fe)

{'Age': 28.66, 'RoomService': 219.27, 'FoodCourt': 439.48, 'ShoppingMall': 177.3, 'Spa': 303.05, 'VRDeck': 310.71, 'PassengerId': '0001_01', 'HomePlanet': 'Earth', 'CryoSleep': False, 'Destination': 'TRAPPIST-1e', 'VIP': False, 'Deck': 'F', 'Side': 'S'}


<font color=green> re 딕셔너리를 수정하고, 결측치 대체 부분부터 코드를 실행하면 됩니당 </font>    

---

In [180]:
# 결측치가 있는 변수에 대체값을 넣어 보정하기
# 데이터프레임.fillna(대체할 값)

for key, value in re.items():
    df_train[key] = df_train[key].fillna(value)

In [181]:
for key, value in fe.items():
    df_test[key] = df_test[key].fillna(value) # test에는 Transported 컬럼이 없고, 사전에는 Transported가 있어서 결측치 정리 불가

In [182]:
df_train.isna().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
Deck            0
Side            0
dtype: int64

In [183]:
df_test.isna().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Deck            0
Side            0
dtype: int64

# 2-2. EDA

## 레이블인코딩 / 원핫인코딩

In [184]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Destination   8693 non-null   object 
 4   Age           8693 non-null   float64
 5   VIP           8693 non-null   bool   
 6   RoomService   8693 non-null   float64
 7   FoodCourt     8693 non-null   float64
 8   ShoppingMall  8693 non-null   float64
 9   Spa           8693 non-null   float64
 10  VRDeck        8693 non-null   float64
 11  Transported   8693 non-null   bool   
 12  Deck          8693 non-null   object 
 13  Side          8693 non-null   object 
dtypes: bool(3), float64(6), object(5)
memory usage: 772.6+ KB


In [185]:
import pandas as pd
import numpy as np


def label_encoding(df_train, df_test, column):
    from sklearn.preprocessing import LabelEncoder

    # labelencoder 생성
    le = LabelEncoder()

    # data 입력 
    le.fit( df_train[column] )                                 

    print("({}) classes: {}".format(column, le.classes_))

    # 적용
    new_column = '{}_encoded'.format(column)
    df_train[new_column] = le.transform( df_train[column] )    
    df_train = df_train.reset_index(drop=True)
    
    if df_test is not None:
        df_test[new_column] = le.transform( df_test[column] )      
        df_test = df_test.reset_index(drop=True)
    else:
        df_test = None
    return df_train, df_test

"""
def onehot_encoding(df_train, df_test, column):
    from sklearn.preprocessing import OneHotEncoder

    # labelencoder 생성
    oe = OneHotEncoder(sparse=False)

    # data 입력
    # (선형회귀와 다른 점): test data가 없더라도 그냥 가~
    if df_test is not None:
        data = pd.concat([df_train[[column]], df_test[[column]]], axis=0).reset_index(drop=True) 
    else:
        data = df_train[[column]]
    oe.fit( data )                                 

    # 적용
    tr_onehot_enc = oe.transform( df_train[[column]] )
    print(tr_onehot_enc.shape)
    
    if df_test is not None:
        te_onehot_enc = oe.transform( df_test[[column]] )
        print(te_onehot_enc.shape)
    
    #columns = oe.categories_[0]
    columns = ['{}_{}'.format(column, i) 
               for i in range(tr_onehot_enc.shape[1])]
    df_onehot_tr = pd.DataFrame(tr_onehot_enc, columns=columns)  
    df_train = pd.concat((df_train.reset_index(drop=True), df_onehot_tr), axis=1)
    
    if df_test is not None:
        df_onehot_te = pd.DataFrame(te_onehot_enc, columns=columns)
        df_test = pd.concat((df_test.reset_index(drop=True), df_onehot_te), axis=1)
    else:
        df_test = None
    return df_train, df_test
"""

"\ndef onehot_encoding(df_train, df_test, column):\n    from sklearn.preprocessing import OneHotEncoder\n\n    # labelencoder 생성\n    oe = OneHotEncoder(sparse=False)\n\n    # data 입력\n    # (선형회귀와 다른 점): test data가 없더라도 그냥 가~\n    if df_test is not None:\n        data = pd.concat([df_train[[column]], df_test[[column]]], axis=0).reset_index(drop=True) \n    else:\n        data = df_train[[column]]\n    oe.fit( data )                                 \n\n    # 적용\n    tr_onehot_enc = oe.transform( df_train[[column]] )\n    print(tr_onehot_enc.shape)\n    \n    if df_test is not None:\n        te_onehot_enc = oe.transform( df_test[[column]] )\n        print(te_onehot_enc.shape)\n    \n    #columns = oe.categories_[0]\n    columns = ['{}_{}'.format(column, i) \n               for i in range(tr_onehot_enc.shape[1])]\n    df_onehot_tr = pd.DataFrame(tr_onehot_enc, columns=columns)  \n    df_train = pd.concat((df_train.reset_index(drop=True), df_onehot_tr), axis=1)\n    \n    if df_test is no

df_train 로 원핫인코딩 하기

In [186]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Destination   8693 non-null   object 
 4   Age           8693 non-null   float64
 5   VIP           8693 non-null   bool   
 6   RoomService   8693 non-null   float64
 7   FoodCourt     8693 non-null   float64
 8   ShoppingMall  8693 non-null   float64
 9   Spa           8693 non-null   float64
 10  VRDeck        8693 non-null   float64
 11  Transported   8693 non-null   bool   
 12  Deck          8693 non-null   object 
 13  Side          8693 non-null   object 
dtypes: bool(3), float64(6), object(5)
memory usage: 772.6+ KB


In [187]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4277 non-null   object 
 2   CryoSleep     4277 non-null   bool   
 3   Destination   4277 non-null   object 
 4   Age           4277 non-null   float64
 5   VIP           4277 non-null   bool   
 6   RoomService   4277 non-null   float64
 7   FoodCourt     4277 non-null   float64
 8   ShoppingMall  4277 non-null   float64
 9   Spa           4277 non-null   float64
 10  VRDeck        4277 non-null   float64
 11  Deck          4277 non-null   object 
 12  Side          4277 non-null   object 
dtypes: bool(2), float64(6), object(5)
memory usage: 376.0+ KB


In [188]:
df_train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Transported', 'Deck', 'Side'],
      dtype='object')

In [189]:
df_train.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Side
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,P
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,S
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,S
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,S
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,S


In [190]:
object_type = ['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']

for column in object_type:
    df_train, _ = label_encoding(df_train, None, column)
    #df_train, _ = onehot_encoding(df_train, None, f'{column}_encoded')

(PassengerId) classes: ['0001_01' '0002_01' '0003_01' ... '9279_01' '9280_01' '9280_02']
(HomePlanet) classes: ['Earth' 'Europa' 'Mars']
(CryoSleep) classes: [False  True]
(Destination) classes: ['55 Cancri e' 'PSO J318.5-22' 'TRAPPIST-1e']
(VIP) classes: [False  True]
(Deck) classes: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'T']
(Side) classes: ['P' 'S']


In [191]:
object_type2 = ['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']

for column in object_type2:
    df_test, _ = label_encoding(df_test, None, column)
    #df_test, _ = onehot_encoding(df_test, None, f'{column}_encoded')

(PassengerId) classes: ['0013_01' '0018_01' '0019_01' ... '9271_01' '9273_01' '9277_01']
(HomePlanet) classes: ['Earth' 'Europa' 'Mars']
(CryoSleep) classes: [False  True]
(Destination) classes: ['55 Cancri e' 'PSO J318.5-22' 'TRAPPIST-1e']
(VIP) classes: [False  True]
(Deck) classes: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'T']
(Side) classes: ['P' 'S']


In [192]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PassengerId          8693 non-null   object 
 1   HomePlanet           8693 non-null   object 
 2   CryoSleep            8693 non-null   bool   
 3   Destination          8693 non-null   object 
 4   Age                  8693 non-null   float64
 5   VIP                  8693 non-null   bool   
 6   RoomService          8693 non-null   float64
 7   FoodCourt            8693 non-null   float64
 8   ShoppingMall         8693 non-null   float64
 9   Spa                  8693 non-null   float64
 10  VRDeck               8693 non-null   float64
 11  Transported          8693 non-null   bool   
 12  Deck                 8693 non-null   object 
 13  Side                 8693 non-null   object 
 14  PassengerId_encoded  8693 non-null   int64  
 15  HomePlanet_encoded   8693 non-null   i

In [193]:
len(df_train.columns)

21

In [194]:
len(df_test.columns)

20

In [195]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PassengerId          4277 non-null   object 
 1   HomePlanet           4277 non-null   object 
 2   CryoSleep            4277 non-null   bool   
 3   Destination          4277 non-null   object 
 4   Age                  4277 non-null   float64
 5   VIP                  4277 non-null   bool   
 6   RoomService          4277 non-null   float64
 7   FoodCourt            4277 non-null   float64
 8   ShoppingMall         4277 non-null   float64
 9   Spa                  4277 non-null   float64
 10  VRDeck               4277 non-null   float64
 11  Deck                 4277 non-null   object 
 12  Side                 4277 non-null   object 
 13  PassengerId_encoded  4277 non-null   int64  
 14  HomePlanet_encoded   4277 non-null   int64  
 15  CryoSleep_encoded    4277 non-null   i

레이블 컬럼 & feature 컬럼 확인

In [198]:
df_train.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,...,Transported,Deck,Side,PassengerId_encoded,HomePlanet_encoded,CryoSleep_encoded,Destination_encoded,VIP_encoded,Deck_encoded,Side_encoded
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,...,False,B,P,0,1,0,2,0,1,0
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,...,True,F,S,1,0,0,2,0,5,1
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,...,False,A,S,2,1,0,2,1,0,1
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,...,False,A,S,3,1,0,2,0,0,1
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,...,True,F,S,4,0,0,2,0,5,1


In [199]:
df_train=df_train.drop(columns='PassengerId')
df_train=df_train.drop(columns='HomePlanet')
df_train=df_train.drop(columns='CryoSleep')
df_train=df_train.drop(columns='Destination')
df_train=df_train.drop(columns='VIP')
df_train=df_train.drop(columns='Deck')
df_train=df_train.drop(columns='Side')

In [200]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  8693 non-null   float64
 1   RoomService          8693 non-null   float64
 2   FoodCourt            8693 non-null   float64
 3   ShoppingMall         8693 non-null   float64
 4   Spa                  8693 non-null   float64
 5   VRDeck               8693 non-null   float64
 6   Transported          8693 non-null   bool   
 7   PassengerId_encoded  8693 non-null   int64  
 8   HomePlanet_encoded   8693 non-null   int64  
 9   CryoSleep_encoded    8693 non-null   int64  
 10  Destination_encoded  8693 non-null   int64  
 11  VIP_encoded          8693 non-null   int64  
 12  Deck_encoded         8693 non-null   int64  
 13  Side_encoded         8693 non-null   int64  
dtypes: bool(1), float64(6), int64(7)
memory usage: 891.5 KB


In [201]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PassengerId          4277 non-null   object 
 1   HomePlanet           4277 non-null   object 
 2   CryoSleep            4277 non-null   bool   
 3   Destination          4277 non-null   object 
 4   Age                  4277 non-null   float64
 5   VIP                  4277 non-null   bool   
 6   RoomService          4277 non-null   float64
 7   FoodCourt            4277 non-null   float64
 8   ShoppingMall         4277 non-null   float64
 9   Spa                  4277 non-null   float64
 10  VRDeck               4277 non-null   float64
 11  Deck                 4277 non-null   object 
 12  Side                 4277 non-null   object 
 13  PassengerId_encoded  4277 non-null   int64  
 14  HomePlanet_encoded   4277 non-null   int64  
 15  CryoSleep_encoded    4277 non-null   i

In [202]:
df_test=df_test.drop(columns='PassengerId')
df_test=df_test.drop(columns='HomePlanet')
df_test=df_test.drop(columns='CryoSleep')
df_test=df_test.drop(columns='Destination')
df_test=df_test.drop(columns='VIP')
df_test=df_test.drop(columns='Deck')
df_test=df_test.drop(columns='Side')

In [203]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  4277 non-null   float64
 1   RoomService          4277 non-null   float64
 2   FoodCourt            4277 non-null   float64
 3   ShoppingMall         4277 non-null   float64
 4   Spa                  4277 non-null   float64
 5   VRDeck               4277 non-null   float64
 6   PassengerId_encoded  4277 non-null   int64  
 7   HomePlanet_encoded   4277 non-null   int64  
 8   CryoSleep_encoded    4277 non-null   int64  
 9   Destination_encoded  4277 non-null   int64  
 10  VIP_encoded          4277 non-null   int64  
 11  Deck_encoded         4277 non-null   int64  
 12  Side_encoded         4277 non-null   int64  
dtypes: float64(6), int64(7)
memory usage: 434.5 KB


In [204]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  8693 non-null   float64
 1   RoomService          8693 non-null   float64
 2   FoodCourt            8693 non-null   float64
 3   ShoppingMall         8693 non-null   float64
 4   Spa                  8693 non-null   float64
 5   VRDeck               8693 non-null   float64
 6   Transported          8693 non-null   bool   
 7   PassengerId_encoded  8693 non-null   int64  
 8   HomePlanet_encoded   8693 non-null   int64  
 9   CryoSleep_encoded    8693 non-null   int64  
 10  Destination_encoded  8693 non-null   int64  
 11  VIP_encoded          8693 non-null   int64  
 12  Deck_encoded         8693 non-null   int64  
 13  Side_encoded         8693 non-null   int64  
dtypes: bool(1), float64(6), int64(7)
memory usage: 891.5 KB


In [206]:
df_train.columns

Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Transported', 'PassengerId_encoded', 'HomePlanet_encoded',
       'CryoSleep_encoded', 'Destination_encoded', 'VIP_encoded',
       'Deck_encoded', 'Side_encoded'],
      dtype='object')

In [208]:
feature_columns =df_test.columns

In [209]:
feature_columns

Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'PassengerId_encoded', 'HomePlanet_encoded', 'CryoSleep_encoded',
       'Destination_encoded', 'VIP_encoded', 'Deck_encoded', 'Side_encoded'],
      dtype='object')

In [210]:
label_columns='Transported'

In [ ]:
"""
feature_columns = df_train.columns
feature_columns """

In [ ]:
#label_columns

In [ ]:
#df_train.info()

In [ ]:
"""object_type3 = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']

for column in object_type3:
    df_train, _ = label_encoding(df_train, None, column)
    df_train, _ = onehot_encoding(df_train, None, f'{column}_encoded')"""

In [ ]:
"""object_type3 = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']

for column in object_type3:
    df_test, _ = label_encoding(df_test, None, column)
    df_test, _ = onehot_encoding(df_test, None, f'{column}_encoded')"""

In [205]:
df_train.isnull().sum()

Age                    0
RoomService            0
FoodCourt              0
ShoppingMall           0
Spa                    0
VRDeck                 0
Transported            0
PassengerId_encoded    0
HomePlanet_encoded     0
CryoSleep_encoded      0
Destination_encoded    0
VIP_encoded            0
Deck_encoded           0
Side_encoded           0
dtype: int64

# 3. 모델 개발

train_test_split

총 2개의 데이터셋을 만들었지만, df_train_1만 모델에 fit할 것임 (df_train_2는 앞으로 이용 X)

In [211]:
# train_test_split
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(
    df_train[feature_columns], df_train[label_columns], stratify=df_train[label_columns], test_size=0.2)

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

print("x_train: {}".format(len(x_train)))
print("x_val: {}".format(len(x_val)))

x_train: 6954
x_val: 1739


로지스틱

In [212]:
# 로지스틱
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

# 모델 학습
model = LogisticRegression(random_state=0) 
model.fit(x_train, y_train)

# 모델 적용
y_pred = model.predict(x_val) 
y_pred_prob = model.predict_proba(x_val) 

# 평가
score_acc = accuracy_score(y_true=y_val, y_pred=y_pred)
score_ll = log_loss(y_true=y_val, y_pred=y_pred)
auc = roc_auc_score(y_true=y_val, y_score=y_pred_prob[:, 1])

print("accuracy: {:.3f}".format(score_acc))
print("logloss: {:.3f}".format(score_ll))
print("auc: {:.3f}".format(auc))

accuracy: 0.797
logloss: 7.011
auc: 0.869


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


KNN

In [213]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

# 모델 학습
k = 5 # hyper-parameter로 k는 5 선정
model = KNeighborsClassifier(n_neighbors=k) 
model.fit(x_train, y_train)

# 모델 적용
y_pred = model.predict(x_val) 
y_pred_prob = model.predict_proba(x_val) 

# 평가
score_acc = accuracy_score(y_true=y_val, y_pred=y_pred)
score_ll = log_loss(y_true=y_val, y_pred=y_pred)
auc = roc_auc_score(y_true=y_val, y_score=y_pred_prob[:, 1])

print("accuracy: {:.3f}".format(score_acc))
print("logloss: {:.3f}".format(score_ll))
print("auc: {:.3f}".format(auc))

accuracy: 0.758
logloss: 8.362
auc: 0.808


의사결정나무

In [214]:
# 의사결정나무
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

# 모델 적용
y_pred = clf.predict(x_val) 
y_pred_prob = clf.predict_proba(x_val) 

# 평가
score_acc = accuracy_score(y_true=y_val, y_pred=y_pred)
score_ll = log_loss(y_true=y_val, y_pred=y_pred)
auc = roc_auc_score(y_true=y_val, y_score=y_pred_prob[:, 1])

print("accuracy: {:.3f}".format(score_acc))
print("logloss: {:.3f}".format(score_ll))
print("auc: {:.3f}".format(auc))

accuracy: 0.745
logloss: 8.819
auc: 0.745


랜덤포레스트

In [215]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(x_train, y_train)

# 모델 적용
y_pred = clf.predict(x_val) 
y_pred_prob = clf.predict_proba(x_val) 

# 평가
score_acc = accuracy_score(y_true=y_val, y_pred=y_pred)
score_ll = log_loss(y_true=y_val, y_pred=y_pred)
auc = roc_auc_score(y_true=y_val, y_score=y_pred_prob[:, 1])

print("accuracy: {:.3f}".format(score_acc))
print("logloss: {:.3f}".format(score_ll))
print("auc: {:.3f}".format(auc))

accuracy: 0.795
logloss: 7.071
auc: 0.888


- Accuracy; 정확도(Accuracy)는 전체 데이터에 대한 예측 오류의 수로 볼 수 있다. 쉽게 말해, 전체 데이터 중에서 몇 개를 맞췄는가이다. 100개 중 97개를 맞췄다면 accuracy는 97%이다.

- Log Loss; 손실(Loss)은 실제 정답과 모델이 예측 한 값 사이의 차이(거리 또는 오차)이다. 즉, 손실이 클수록 데이터에 대한 오류도 커진다. 쉽게 말해, 틀리게 예측한 경우 얼마나 오류를 범했는가로 볼 수 있다. 따라서 2개의 모델이 100개 중 97개를 똑같이 맞춰도 둘의 오차는 다를 수 있다.

- classification 모델평가시 Log Loss를 사용하지만, 회귀분석 모델에서는 MSE(Mean Squared Error)를 loss function으로 사용함 why? 로지스틱 회귀에서는 Non-convex한 경우 지역해에 빠질 위험이 있음

앙상블모형

In [216]:
# 앙상블 모형
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report



# 서로 다른 3개의 모델을 가지고 각각 hetero하게 학습시킨 다음에 결과물에 대해 voting을 진행함
model_lr = LogisticRegression(random_state=999, solver='liblinear')
model_knn = KNeighborsClassifier(n_neighbors=5) # hyper-parameter
model_dt = DecisionTreeClassifier(criterion='entropy', random_state=999)

model_ensemble = VotingClassifier(
    estimators=[('lr', model_lr), ('knn', model_knn), ('dt', model_dt)],
    voting='soft'
)

# VotingClassifier
# estimators=[('lr', model_lr), ('knn', model_knn), ('dt', model_dt)] -> 각각 estimator에 pair로 넣어줌
# voting = 'soft' or 'hard'

## train 
model_ensemble.fit(x_train, y_train)

## eval (test)
y_pred = model_ensemble.predict(x_val)
y_pred_prob = model_ensemble.predict_proba(x_val)

score_acc = accuracy_score(y_true=y_val, y_pred=y_pred)
#score_acc = model_ensemble.score(x_test, y_test)  # mean_accuracy를 return 해줍니다
score_auc = roc_auc_score(y_true=y_val, y_score=y_pred_prob[:, 1])

print("(ensemble) acc: {:.3f}, auc: {:.3f}".format(score_acc, score_auc))

(ensemble) acc: 0.783, auc: 0.871


In [217]:
# 각각 세개 모델 따로따로 넣어서 해주지 않고, for문 돌려서 각 모델마다 산출하게 함

for model_name, model in [('lr', model_lr), ('knn', model_knn), ('dt', model_dt)]:
    model.fit(x_train, y_train)
    y_pred = model.predict(x_val)
    y_pred_prob = model.predict_proba(x_val)
    score_acc = accuracy_score(y_true=y_val, y_pred=y_pred)
    score_auc = roc_auc_score(y_true=y_val, y_score=y_pred_prob[:, 1])
    print("(model_{}) acc: {:.3f}, auc: {:.3f}".format(model_name, score_acc, score_auc))
    

(model_lr) acc: 0.795, auc: 0.887
(model_knn) acc: 0.758, auc: 0.808
(model_dt) acc: 0.742, auc: 0.742


<GBM과 변주모델들 개요>
(1) Gradient Boosting (GBM)
- 예측력이 좋지 못한 데이터/변수들을 복제해서 
     예측력이 약했던 데이터/변수들의 성능을 높이는 모델
- 어떻게 노드를 분할할 것인가에 대한 경우의 수를 다 고려
- (단) 속도가 느릴 수 있다, 하드웨어의 성능에 민감

(2) XGBoost
- 노드 탐색을 할 때 모든 경우의 수를 고려 X
- (1) Approximation Algorithm
- 데이터 분포---> 데이터를 퍼센타일로 나눔--->버킷 안에서 노드분할 split 지점을 고려 (하나의 변수 안에서 모든 데이터 고려X, 하나의 변수 안에서 퍼센타일로 버킷 나눈 후 split 지점 고려)
- (2) Column Sampling
- 트리를 생성할 때, 변수를 임의적으로 샘플링 해서 사용

(3) LightGBM
- 변수가 너무 많아서 데이터가 클 때 
- (1) 단측 표본 추출법
- 데이터의 개수를 줄임
- 예측성능이 우수한 데이터들은 살려둠
- 예측성능이 우수하지 않았던 데이터(약한 기울기를 가진 데이터)--> 랜덤 샘플
데이터 비워진 것들을 우수한/약했던 데이터들로 복제/생성
- (2) 배타적 변수 추출
- (one hot encoding) 배타적 변수들을 단일변수 묶음으로 묶어 취급해 모델링에 사용
- GBM 장점 O. 단점 보완

XG Boost

In [218]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
import pandas as pd


## model
model_xgb = xgb.XGBClassifier(
    objective="binary:logistic", 
    eval_metric='auc',
    random_state=999,
)

## train 
model_xgb.fit(x_train, y_train)

## test
y_pred_prob = model_xgb.predict_proba(x_val)
score = roc_auc_score(y_true=y_val, y_score=y_pred_prob[:, 1])

print("(test) score: {:.3f}".format(score))

(test) score: 0.902


Light GBM

In [219]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, train_test_split
import pandas as pd

## model
model_lgb = lgb.LGBMClassifier(
    objective="binary", random_state=999,
)

## train 
model_lgb.fit(x_train, y_train)

## test
y_pred_prob = model_lgb.predict_proba(x_val)
score = roc_auc_score(y_true=y_val, y_score=y_pred_prob[:, 1])

print("(test) score: {:.3f}".format(score))

(test) score: 0.904


K-fold 검증기법, Cross Validation

### 어떤 것을 최종모델로 결정할까요?
- Light GBM

# 5. 제출 데이터 생성

sample_submission.csv - 제출 파일 형식의 예시입니다.

- PassengerId : 각 승객에 대한 ID입니다.
- Transported : 목표값 입니다. 승객에 따라True 또는 False 를 예측합니다.

In [ ]:
## inference
#x_test = df_test_clean[features].values
#df_test_clean['Survived'] = best_model.predict(x_test)

## dump
#df_test_clean[['PassengerId', 'Survived']].to_csv('./submission.csv', index=False)

In [ ]:
#df_sub = pd.read_csv('./sample_submission.csv')
#df_sub['Transported'] = result
#df_sub.to_csv('./submission.csv', index=False)

In [220]:
d = model_lgb.predict(df_test)
da_lgb = pd.DataFrame(d)

# da_lgb
da_lgb.columns = ['Transported']
da_lgb['PassengerId'] = id
da_lgb['kk'] = da_lgb['Transported']
da_lgb = da_lgb.drop(['Transported'],axis=1)
da_lgb.columns = ['PassengerId','Transported']
da_lgb['Transported'] = da_lgb['Transported'].apply(lambda x:True if x==1 else False)
# da_lgb = da_lgb.set_index('PassengerId')

print(da_lgb)

## dump
da_lgb.to_csv('./submission3.csv', index=False)

                 PassengerId  Transported
0     <built-in function id>         True
1     <built-in function id>        False
2     <built-in function id>         True
3     <built-in function id>         True
4     <built-in function id>         True
...                      ...          ...
4272  <built-in function id>         True
4273  <built-in function id>         True
4274  <built-in function id>         True
4275  <built-in function id>         True
4276  <built-in function id>         True

[4277 rows x 2 columns]


## [!] submit results

In [221]:
! kaggle competitions submit -c spaceship-titanic -f submission.csv -m "GSI7645.01-00__week03"

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/kaggle/cli.py", line 67, in main
    out = args.func(**command_args)
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 562, in competition_submit_cli
    competition, quiet)
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 513, in competition_submit
    content_length=os.path.getsize(file_name),
  File "/usr/lib/python3.7/genericpath.py", line 50, in getsize
    return os.stat(filename).st_size
FileNotFoundError: [Errno 2] No such file or directory: 'submission.csv'


In [ ]:
! kaggle competitions submissions -c spaceship-titanic